<table class="tfo-notebook-buttons" align="left">
<td style="border: none;">
<a target="blank" href="https://colab.research.google.com/github/Hari31416/Portfolio/blob/main/ML/Housing_Price/Preprocessing.ipynb?hl=en"><img src="https://colab.research.google.com/img/colab_favicon_256px.png" width="28" height="28"/>Run on Google Colab</a></td>
<td style="border: none;">
<a target="blank" href="https://github.com/Hari31416/Portfolio/blob/main/Housing_Price/ML/Preprocessing.ipynb"><img src="https://cdn.icon-icons.com/icons2/2368/PNG/512/github_logo_icon_143772.png" width="28" height="28"/>View on Github</a></td>
<td style="border: none;">
</table>

## Imports

In [217]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, StandardScaler

## Loading Data

In [218]:
test_c = pd.read_csv("data/test_cleaned.csv")
train_c = pd.read_csv("data/train_cleaned.csv")

## Outliers

During EDA, we found that the data contains some outliers. We can remove them from the train dataset. However, as we need to make prediction for all the entries in the test dataset, we can not remove them from the test dataset. 

## Preprocessing

### Functions

In [220]:
def get_detail(column):
    """ Get the detail of the column 
    
    Args:
        column (str): the column name
    
    Returns:
        str: the detail of the column
    """
    text = ""
    with open("data_description.txt") as f:
        text = f.read()
    
    try:
        start = text.find(f"{column}:")
        end = text.find(":", start+len(column)+5)
        texts_found = text[start:end].strip().split("\n")
        print("\n".join(texts_found[:-2]))
    except:
        print("No such column")

In [ ]:
str_cols = train_c.columns[train_c.dtypes == 'object']
str_cols = list(str_cols)

### Should We Use All The Columns?

A ton of non-numerical columns are in the dataset. If we use all of them, the resulting dataframe after we have converted the non-numerical columns to numerical columns will be very large. So, we need to take only a few of these columns.

In [221]:
train_c_str = train_c[str_cols]
test_c_str = test_c[str_cols]
train_c_nums = train_c.drop(str_cols, axis=1)
test_c_nums = test_c.drop(str_cols, axis=1)

In [222]:
ohe = OneHotEncoder(sparse=False)
train_c_str_ohe = ohe.fit_transform(train_c_str)
test_c_str_ohe = ohe.transform(test_c_str)


In [223]:
train_c_str_ohe.shape, test_c_str_ohe.shape

((1460, 266), (1459, 266))

As we can see, after one-hot encoding, the dataset contains 266 columns just for the non-numeric columns. 

### Which columns should we use?

#### Using Ranking System

Let's have a look on the columns and what they specify.

In [224]:
for col in str_cols:
    get_detail(col)
    print("\n")
    print("**********"*5)
    print("\n")

MSZoning: Identifies the general zoning classification of the sale.
		
       A	Agriculture
       C	Commercial
       FV	Floating Village Residential
       I	Industrial
       RH	Residential High Density
       RL	Residential Low Density
       RP	Residential Low Density Park 
       RM	Residential Medium Density


**************************************************


Street: Type of road access to property

       Grvl	Gravel	
       Pave	Paved


**************************************************


Alley: Type of alley access to property

       Grvl	Gravel
       Pave	Paved
       NA 	No alley access


**************************************************


LotShape: General shape of property

       Reg	Regular	
       IR1	Slightly irregular
       IR2	Moderately Irregular
       IR3	Irregular


**************************************************


LandContour: Flatness of the property

       Lvl	Near Flat/Level	
       Bnk	Banked - Quick and significant rise from street grade to buil

There are a lot of columns where comparative adjectives like "excellent", "good", "bad" are used. Since these adjectives give a sense of 'ranking', we can convert them to numerical values. The encoding, we'll use is as follows:

Ex	Excellent >> 5

Gd	Good >> 4

TA	Typical - slight dampness allowed >> 3

Fa	Fair - dampness or some cracking or settling >> 2

Po	Poor - Severe cracking, settling, or wetness >> 1

NA	No Basement >> 0


In BsmtFinType columns, we have an extra adjective "Unf". This adjective is used to indicate that the basement is not finished. We'll merge it to `NA`.

List of columns which can be transformed to numerical values by the method described above are: `"ExterQual, ExterCond, BsmtQual, BsmtCond, BsmtExposure, BsmtFinType1, BsmtFinType2,  HeatingQC, KitchenQual, FireplaceQu, GarageQual, GarageCond, PoolQC, Fence"`.

Apart from these, we also have some columns which don't give a direct sense of 'ranking' but we can, instead of dropping them, convert them to numerical values using a generalization of the method described above. These columns are: `"LotShape", "Utilities", "GarageFinish", "LandContour", "LandSlope", "CentralAir", "PavedDrive"`. Let's add these columns too.

In [225]:
cols = "ExterQual, ExterCond, BsmtQual, BsmtCond, BsmtExposure, BsmtFinType1, BsmtFinType2, HeatingQC, KitchenQual, FireplaceQu, GarageQual, GarageCond, PoolQC, Fence, LotShape, Utilities, GarageFinish, LandContour, LandSlope, CentralAir, PavedDrive".split(", ")
cols

['ExterQual',
 'ExterCond',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinType2',
 'HeatingQC',
 'KitchenQual',
 'FireplaceQu',
 'GarageQual',
 'GarageCond',
 'PoolQC',
 'Fence',
 'LotShape',
 'Utilities',
 'GarageFinish',
 'LandContour',
 'LandSlope',
 'CentralAir',
 'PavedDrive']

Next we make a function to convert these columns to numerical values.

In [226]:
zero = ["NA", "Unf", "None", "No"]
one = ["Po", "No", "LwQ", "IR3", "ELO", "Low", "Sev", "Yes", "N"]
two = ["Fa", "Mn", "Rec", "MnWw", "IR2", "Unf", "HLS"]
three = ["TA", "BLQ", "GdWo", "IR1", "NoSeWa", "RFn", "Bnk", "Mod","P"]
four = ["Gd", "ALQ", "Av", "MnPrv", "NoSewr"]
five = ["Ex", "GLQ", "GdPrv", "Reg", "AllPub", "Fin", "Lvl", "Gtl", "Y"]

num_dict = {
    0:zero,
    1:one,
    2:two,
    3:three,
    4:four,
    5:five
}

def make_numeric(column, data):
    """ Make the column numeric
    
    Args:
        column (str): the column name
        data (pandas.DataFrame): the dataframe
    
    Returns:
        pandas.DataFrame: the dataframe with the column numeric
    """
    
    for key, value in num_dict.items():
        for val in value:
            data[column] = data[column].replace(val, key)
    data[column] = data[column].astype(int)
    return data
    

In [228]:
data = train_c_str.copy()
for col in cols:
    data = make_numeric(col, data)

In [229]:
for col in cols:
    print(col, data[col].dtype)

ExterQual int32
ExterCond int32
BsmtQual int32
BsmtCond int32
BsmtExposure int32
BsmtFinType1 int32
BsmtFinType2 int32
HeatingQC int32
KitchenQual int32
FireplaceQu int32
GarageQual int32
GarageCond int32
PoolQC int32
Fence int32
LotShape int32
Utilities int32
GarageFinish int32
LandContour int32
LandSlope int32
CentralAir int32
PavedDrive int32


In [230]:
train_c_nums = pd.concat([train_c_nums, data[cols]], axis=1)
(train_c_nums.dtypes == "object").sum()

0

Great! Let's do the same for train dataset.

In [231]:
data = test_c_str.copy()
for col in cols:
    data = make_numeric(col, data)

In [232]:
for col in cols:
    print(col, data[col].dtype)

ExterQual int32
ExterCond int32
BsmtQual int32
BsmtCond int32
BsmtExposure int32
BsmtFinType1 int32
BsmtFinType2 int32
HeatingQC int32
KitchenQual int32
FireplaceQu int32
GarageQual int32
GarageCond int32
PoolQC int32
Fence int32
LotShape int32
Utilities int32
GarageFinish int32
LandContour int32
LandSlope int32
CentralAir int32
PavedDrive int32


In [233]:
test_c_nums = pd.concat([test_c_nums, data[cols]], axis=1)
(test_c_nums.dtypes == "object").sum()

0

We'll remove these columns from the non-numerical columns list.

In [234]:
train_c_str = train_c_str.drop(cols, axis=1)
test_c_str = test_c_str.drop(cols, axis=1)
train_c_str.columns

Index(['MSZoning', 'Street', 'Alley', 'LotConfig', 'Neighborhood',
       'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle',
       'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'Foundation',
       'Heating', 'Electrical', 'Functional', 'GarageType', 'MiscFeature',
       'SaleType', 'SaleCondition'],
      dtype='object')

In [235]:
len(train_c_str.columns)

22

#### The Remaing Columns

This still leaves us with 22 columns. Some of them have a lot of categories in them. What should we do with it? Well, I'm going to drop almost all of them, except some of them.

In [236]:
not_to_drop = ["MSZoning", "Neighborhood", "SaleType", "SaleCondition"]
to_drop = []
for col in train_c_str.columns:
    if col not in not_to_drop:
        to_drop.append(col)

In [237]:
train_c_str = train_c_str.drop(to_drop, axis=1)
test_c_str = test_c_str.drop(to_drop, axis=1)
train_c_str.shape

(1460, 4)

Next, we'll use pandas `get_dummy` to convert these to numerical values.

In [238]:
train_c_str = pd.get_dummies(train_c_str, drop_first=True)
test_c_str = pd.get_dummies(test_c_str, drop_first=True)

Finally, concatenate these.

In [239]:
train_c_nums = pd.concat([train_c_nums, train_c_str], axis=1)
test_c_nums = pd.concat([test_c_nums, test_c_str], axis=1)

In [240]:
train_c_nums.shape, test_c_nums.shape

((1460, 100), (1459, 99))

Let's check whether all the columns are numerical or not.

In [241]:
(train_c_nums.dtypes == "object").sum()

0

In [242]:
(test_c_nums.dtypes == "object").sum()

0

So, all the columns are numerical. Next, we need to standardize them. But first, we have to drop some of the columns as they don't give any information.

### Dropping Irrelevant Columns

#### ID Column

In [243]:
train_Id = train_c_nums["Id"]
test_Id = test_c_nums["Id"]

In [261]:
train_Id.to_csv("data/train_Id.csv", index=False)
test_Id.to_csv("data/test_Id.csv", index=False)

In [244]:
train_c_nums = train_c_nums.drop(["Id"], axis=1)
test_c_nums = test_c_nums.drop(["Id"], axis=1)

#### Year Columns

In [245]:
years_cols = ["YearBuilt", "YearRemodAdd", "GarageYrBlt"]
train_c_nums = train_c_nums.drop(years_cols, axis=1)
test_c_nums = test_c_nums.drop(years_cols, axis=1)

#### The Target Column

In [246]:
X = train_c_nums.drop("SalePrice", axis=1)
y = train_c_nums["SalePrice"]

In [262]:
y.to_csv("data/y.csv", index=False)

In [247]:
assert X.shape[1] == test_c_nums.shape[1] ,"Number of columns are not equal"

### Normalizing the Data

We don't need to standardize all the columns, in fact, some of the columns must not be standardized. We'll standardize only those columns which have a huge range, say greater than 10.

In [248]:
def get_range(column, data):
    """ Get the range of the column
    
    Args:
        column (str): the column name
        data (pandas.DataFrame): the dataframe
    
    Returns:
        tuple: the range of the column
    """
    
    return data[column].max() - data[column].min()

In [249]:
large_range = []
for col in X.columns:
    range_ = get_range(col, X)
    if range_>10:
        large_range.append(col)
large_range

['MSSubClass',
 'LotFrontage',
 'LotArea',
 'MasVnrArea',
 'BsmtFinSF1',
 'BsmtFinSF2',
 'BsmtUnfSF',
 'TotalBsmtSF',
 '1stFlrSF',
 '2ndFlrSF',
 'LowQualFinSF',
 'GrLivArea',
 'TotRmsAbvGrd',
 'GarageArea',
 'WoodDeckSF',
 'OpenPorchSF',
 'EnclosedPorch',
 '3SsnPorch',
 'ScreenPorch',
 'PoolArea',
 'MiscVal',
 'MoSold']

We need to normalize only these columns.

In [250]:
X_to_standardize = X[large_range]
test_to_standardize = test_c_nums[large_range]

X_done = X.drop(large_range, axis=1)
test_done = test_c_nums.drop(large_range, axis=1)

In [251]:
test_to_standardize.shape, test_done.shape

((1459, 22), (1459, 73))

In [252]:
X_to_standardize.shape, X_done.shape

((1460, 22), (1460, 73))

Using `StandardScaler` from `sklearn.preprocessing` we can standardize the data.

In [253]:
scaler = StandardScaler()
scaler.fit(X_to_standardize)
X_standard = scaler.transform(X_to_standardize)

scaler.fit(test_to_standardize)
test_standard = scaler.transform(test_to_standardize)

In [254]:
X_standard = pd.DataFrame(X_standard, columns=X_to_standardize.columns)
test_standard = pd.DataFrame(test_standard, columns=test_to_standardize.columns)

In [255]:
X = pd.concat([X_done, X_standard], axis=1)
test_c_nums = pd.concat([test_done, test_standard], axis=1)

In [256]:
X.shape, test_c_nums.shape

((1460, 95), (1459, 95))

In [257]:
X.describe()

,OverallQual,OverallCond,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,Fireplaces,GarageCars,...,TotRmsAbvGrd,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold
count,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,...,1.460000e+03,1.460000e+03,1.460000e+03,1.460000e+03,1.460000e+03,1.460000e+03,1.460000e+03,1.460000e+03,1.460000e+03,1.460000e+03
mean,6.099315,5.575342,0.425342,0.057534,1.565068,0.382877,2.866438,1.046575,0.613014,1.767123,...,4.387662e-17,-2.022735e-17,2.190029e-16,3.357284e-17,1.379034e-16,-4.005738e-16,1.199193e-16,-7.945889e-16,2.409602e-16,-6.395189e-17
std,1.382997,1.112799,0.518911,0.238753,0.550916,0.502885,0.815778,0.220338,0.644666,0.747315,...,1.000343e+00,1.000343e+00,1.000343e+00,1.000343e+00,1.000343e+00,1.000343e+00,1.000343e+00,1.000343e+00,1.000343e+00,1.000343e+00
min,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,-2.780469e+00,-2.212963e+00,-7.521758e-01,-7.044833e-01,-3.593249e-01,-1.163393e-01,-2.702084e-01,-6.869175e-02,-8.768781e-02,-1.969111e+00
25%,5.000000,5.000000,0.000000,0.000000,1.000000,0.000000,2.000000,1.000000,0.000000,1.000000,...,-9.341298e-01,-6.479160e-01,-7.521758e-01,-7.044833e-01,-3.593249e-01,-1.163393e-01,-2.702084e-01,-6.869175e-02,-8.768781e-02,-4.891101e-01
50%,6.000000,5.000000,0.000000,0.000000,2.000000,0.000000,3.000000,1.000000,1.000000,2.000000,...,-3.186833e-01,3.284429e-02,-7.521758e-01,-3.270298e-01,-3.593249e-01,-1.163393e-01,-2.702084e-01,-6.869175e-02,-8.768781e-02,-1.191097e-01
75%,7.000000,6.000000,1.000000,0.000000,2.000000,1.000000,3.000000,1.000000,1.000000,2.000000,...,2.967633e-01,4.820057e-01,5.886506e-01,3.221901e-01,-3.593249e-01,-1.163393e-01,-2.702084e-01,-6.869175e-02,-8.768781e-02,6.208910e-01
max,10.000000,9.000000,3.000000,2.000000,3.000000,2.000000,8.000000,3.000000,3.000000,4.000000,...,4.604889e+00,4.421526e+00,6.087635e+00,7.554198e+00,8.675309e+00,1.721723e+01,8.341462e+00,1.830618e+01,3.116527e+01,2.100892e+00


Okay, this should be all!

In [260]:
X.to_csv("data/train_final.csv", index=False)
test_c_nums.to_csv("data/test_final.csv", index=False)